<a href="https://colab.research.google.com/github/Raktim-Srivastava/Uber_reviews_textanalytics/blob/main/TA_Group_Assignment_latest_raktim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

### 0. Understanding the Business Problem
Uber Inc in the US wants to know:

- the major complaints premium users have about their cab services,
- how these impact service ratings.

We as (technical) consultants to Uber. have to:  
- [a] analyze text reviews of Uber cabs’ US services,  
- [b] relate whether and which different features of these reviews impact overall ratings  
- [c] pinpoint possible areas of improvement.

### 1. Pre-processing: 
- Examine the dataset. 
- ID the columns of interest. 
- Drop special characters, html junk etc. 
- Perform any other preprocessing and text-cleaning activity you think fits this context.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Kenrich005/Uber_reviews_textanalytics/main/uber_reviews_itune.csv",
                 encoding='cp1252')
df.head()

,Author_Name,Title,Author_URL,App_Version,Rating,Review,Date
0,#NEVERUBER,Dishonest and Disgusting,https://itunes.apple.com/us/reviews/id663331949,3.434.10005,1,"For half an hour, we tried EVERY UBER SERVICE ...",29-12-2020 01:14
1,$$Heaven,Free offer,https://itunes.apple.com/us/reviews/id810421958,3.434.10005,2,If I’m not eligible for the offer Stop floodin...,01-01-2021 23:17
2,.Disappointed....,Inaccurate,https://itunes.apple.com/us/reviews/id49598333,3.439.10000,2,Consistently inaccurate Uber Eats ETA and the ...,15-01-2021 23:38
3,.i. andrea,bad,https://itunes.apple.com/us/reviews/id689880334,3.434.10005,1,i had my rides canceled back to back. they the...,08-12-2020 01:01
4,-:deka:-,Double charged me for an order,https://itunes.apple.com/us/reviews/id124963835,3.434.10005,1,Two of the same orders was added by accident. ...,15-12-2020 04:02


Columns of interest:  
1. Title - Brief summary about the review
2. Rating - Label for supervised learning
3. Review - To extract the sentiment of the complaint
4. Date - Extracting weekday or weekend may give better insight on nature of review

### Data Cleaning

In [3]:
df1 = df.drop(['Author_Name','Author_URL','App_Version'],axis=1)
df1.head()

,Title,Rating,Review,Date
0,Dishonest and Disgusting,1,"For half an hour, we tried EVERY UBER SERVICE ...",29-12-2020 01:14
1,Free offer,2,If I’m not eligible for the offer Stop floodin...,01-01-2021 23:17
2,Inaccurate,2,Consistently inaccurate Uber Eats ETA and the ...,15-01-2021 23:38
3,bad,1,i had my rides canceled back to back. they the...,08-12-2020 01:01
4,Double charged me for an order,1,Two of the same orders was added by accident. ...,15-12-2020 04:02


## Translate the languages to English.

In [4]:
#!pip install googletrans==3.1.0a0
import googletrans
from googletrans import *


translator = googletrans.Translator()

try:
  df1['Review'] = df1['Review'].astype(str)
  df1['Translated_Review'] = df1['Review'].apply(translator.translate, src='auto', dest='en').apply(getattr, args=('text',))
except AttributeError:
  pass

df1

,Title,Rating,Review,Date,Translated_Review
0,Dishonest and Disgusting,1,"For half an hour, we tried EVERY UBER SERVICE ...",29-12-2020 01:14,"For half an hour, we tried EVERY UBER SERVICE ..."
1,Free offer,2,If I’m not eligible for the offer Stop floodin...,01-01-2021 23:17,If I’m not eligible for the offer Stop floodin...
2,Inaccurate,2,Consistently inaccurate Uber Eats ETA and the ...,15-01-2021 23:38,Consistently inaccurate Uber Eats ETA and the ...
3,bad,1,i had my rides canceled back to back. they the...,08-12-2020 01:01,i had my rides canceled back to back. they the...
4,Double charged me for an order,1,Two of the same orders was added by accident. ...,15-12-2020 04:02,Two of the same orders was added by accident. ...
...,...,...,...,...,...
485,Uber,5,Perdí mi cuenta no la puedo recuperar la use e...,16-01-2021 02:39,I lost my account I can't get it back I used i...
486,Crap crap crap,1,Still the same. I was forced to use it in Colo...,23-12-2020 00:15,Still the same. I was forced to use it in Colo...
487,Sleeping Drivers,1,It is a 30 minute commute from my household to...,16-12-2020 19:10,It is a 30 minute commute from my household to...
488,Bad design re: offer code redemption and issue...,1,Was sent a $30 off UBer Eats. I thought about ...,25-11-2020 23:06,Was sent a $30 off UBer Eats. I thought about ...


In [37]:
# Replacing emoticon with its respective meaning
df_emojis = pd.read_csv("https://raw.githubusercontent.com/Kenrich005/Uber_reviews_textanalytics/main/emoji_description.csv")
df_emojis['Code'] = df_emojis['Code'].str.replace('+','+000')
df_emojis.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Code,CLDR Short Name
0,<U+0001F600>,grinning face
1,<U+0001F603>,grinning face with big eyes
2,<U+0001F604>,grinning face with smiling eyes
3,<U+0001F601>,beaming face with smiling eyes
4,<U+0001F606>,grinning squinting face


In [38]:
df_emojis

,Code,CLDR Short Name
0,<U+0001F600>,grinning face
1,<U+0001F603>,grinning face with big eyes
2,<U+0001F604>,grinning face with smiling eyes
3,<U+0001F601>,beaming face with smiling eyes
4,<U+0001F606>,grinning squinting face
...,...,...
2101,<subdivision-flag>,subdivision-flag
2102,<Code>,CLDR Short Name
2103,<U+0001F3F4 U+000E0067 U+000E0062 U+000E0065 U...,flag: England
2104,<U+0001F3F4 U+000E0067 U+000E0062 U+000E0073 U...,flag: Scotland


In [39]:
# Replacing emoticon with its respective meaning
to_replace = df_emojis.Code.tolist()
replace_with = df_emojis['CLDR Short Name'].tolist()

# using zip() to convert lists to dictionary
res = dict(zip(to_replace, replace_with))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(str(i), j + " ")
    return text

In [40]:
### check following indexes for emoticon replacement
## [1, 37, 89, 132, 145, 190, 208, 214, 237, 357, 367, 439 ]
## index - 439,145 didnot get replaced -- cant find the emojis to replace
###

In [41]:
df1.Review = df1.Review.apply(lambda text: replace_all(text, res))
df1.Review[1]

'If I’m not eligible for the offer Stop flooding my email with this false information pouting face pouting face pouting face '

In [42]:
# df1.Review[37]

In [43]:
# replace_all(df.Review[89],res)

In [44]:
df1.Review = df1.Review.str.split('<').str[0]
df1.shape

(490, 4)

In [45]:
df1.Review[1]

'If I’m not eligible for the offer Stop flooding my email with this false information pouting face pouting face pouting face '

In [46]:
df1['Review'].replace('', np.nan, inplace=True)
df1.dropna(subset=['Review'], inplace=True)
df1.shape

(489, 4)

In [47]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# define unit func to process one doc
from nltk import sent_tokenize, word_tokenize
def vader_unit_func(doc0,column_name):
    sents_list0 = sent_tokenize(doc0)
    vs_doc0 = []
    sent_ind = []
    for i in range(len(sents_list0)):
        vs_sent0 = analyzer.polarity_scores(sents_list0[i])
        vs_doc0.append(vs_sent0)
        sent_ind.append(i)
        
    # obtain output as DF    
    doc0_df = pd.DataFrame(vs_doc0)
    doc0_df.columns = [x+column_name for x in doc0_df.columns]
    doc0_df.insert(0, 'sent_index', sent_ind)  # insert sent index
    doc0_df.insert(doc0_df.shape[1], 'sentence', sents_list0)
    return(doc0_df)

# define wrapper func
def vader_wrap_func(corpus0,column_name):
    
    # use ifinstance() to check & convert input to DF
    if isinstance(corpus0, list):
        corpus0 = pd.DataFrame({'text':corpus0})
    
    # define empty DF to concat unit func output to
    vs_df = pd.DataFrame()    
    
    # apply unit-func to each doc & loop over all docs
    for i1 in range(len(corpus0)):
        doc0 = str(corpus0.iloc[i1])
        vs_doc_df = vader_unit_func(doc0,column_name)  # applying unit-func
        vs_doc_df.insert(0, 'doc_index', i1)  # inserting doc index
        vs_df = pd.concat([vs_df, vs_doc_df], axis=0)
        
    return(vs_df)

ModuleNotFoundError: ignored

In [ ]:
# test-drive wrapper func
review_sentiment = vader_wrap_func(df1.Review,'_review').groupby('doc_index').sum()
title_sentiment = vader_wrap_func(df1.Title,'_title').groupby('doc_index').sum()
df1 = pd.concat([df1,review_sentiment,title_sentiment],axis=1)
print(df1.shape)
df1.head()

In [ ]:
# Converting Date into datetime format
df1['Date'] =  pd.to_datetime(df1['Date'], format='%d-%m-%Y %H:%M')
df1.Date.head()

In [ ]:
df1['Isweekend'] = np.where(df1.Date.dt.dayofweek>4,1,0)
df1['Late_night'] = np.where(df1.Date.dt.hour<4,1,0)
df1['Early_mrng'] = np.where(df1.Date.dt.hour.between(4,8),1,0)
df1['Morning'] = np.where(df1.Date.dt.hour.between(8,12),1,0)
df1['Noon'] = np.where(df1.Date.dt.hour.between(12,16),1,0)
df1['Eve'] = np.where(df1.Date.dt.hour.between(16,20),1,0)
df1['Night'] = np.where(df1.Date.dt.hour>20,1,0)
df1.head()

In [ ]:
df1=df1.drop(['sent_index','Title','Review','Date'],axis=1)
df1.head()

In [ ]:
# Removing null values
df1.dropna(inplace=True)

### Preliminary Regression Model


In [ ]:
y = df1.Rating
X = df1.drop('Rating', axis=1)
y.shape, X.shape

In [ ]:
X.isnull().sum()

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
X.drop('const',axis=1,inplace=True)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Calculating VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):
   
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'].sort_values()

    return(vif)

In [ ]:
calc_vif(X)

Generally, a VIF above 4 or tolerance below 0.25 indicates that multicollinearity might exist, and further investigation is required.   
When VIF is higher than 10 or tolerance is lower than 0.1, there is significant multicollinearity that needs to be corrected.  
  
Since all the above variables have VIF below 4 and above 0.25, we can be assured that there is no multicollinearity.

In [ ]:
# We will save the model performance metrics in a DataFrame

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
Model = []
RMSE = []
R_sq = []
cv = KFold(5)

#Creating a Function to append the cross validation scores of the algorithms
def input_scores(name, model, x, y):
    Model.append(name)
    RMSE.append(np.sqrt((-1) * cross_val_score(model, x, y, cv=cv, scoring='neg_mean_squared_error').mean()))
    R_sq.append(cross_val_score(model, x, y, cv=cv, scoring='r2').mean())

In [ ]:
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor)

names = ['Linear Regression', 'Ridge Regression', 'Lasso Regression',
         'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor',
         'Adaboost Regressor','XGBRegressor']

models = [LinearRegression(), Ridge(), Lasso(),
          KNeighborsRegressor(), DecisionTreeRegressor(),
          RandomForestRegressor(), GradientBoostingRegressor(), 
          AdaBoostRegressor(),XGBRegressor()]

#Running all algorithms
for name, model in zip(names, models):
    input_scores(name, model, X_train, y_train)

Reference: https://www.kaggle.com/swatisinghalmav/best-of-8-regression-models-to-predict-strength

In [ ]:
evaluation = pd.DataFrame({'Model': Model,'RMSE': RMSE,'R Squared': R_sq})
print("FOLLOWING ARE THE TRAINING SCORES: ")
evaluation

## Next Steps:
1. Convert non-English reviews to English or use non-english dictionary - ## need to add to current code
2. Scale the emoticons replacement - ## validation pending
3. Make sentiment analysis of Title - ## Done
4. From Date, extract weekend, weekday, morning, afternoon, evening, night - ## Done
5. Make preliminary regression model with y variable as Ratings - ## Done
6. ?Use OLS - ## Done
7. Feature Engineering - columns on specific word count
8. Shiny App
9. Add PCA